In [9]:
import pandas as pd
import numpy as np
import datetime
from pathlib import Path


# 데이터 Load

In [10]:
# 데이터 경로
path = Path.cwd().parent
path = str(path)+'/GFMS DB/'
path

'/Users/keumhyeonjun/Projects/행안부/자재/GFMS DB/'

In [11]:
FMSA10_작업지시_결과 = pd.read_csv(path+'FMSA10_작업지시_결과_ver2.csv')

FMSA11_작업장비이력 = pd.read_excel(path + 'FMSA11_작업장비이력.xlsx')

FMSX05_장비마스터 = pd.read_csv(path + 'FMSX05_장비마스터_ver2.csv')

FMSB07_자재출고 = pd.read_excel(path + 'FMSB07_자재출고.xlsx')

FMSB03_자재마스터 = pd.read_csv(path + 'FMSB03_자재마스터_ver2.csv')

FMSB03_자재마스터.drop(['class_nm_4','class_cd_4'],axis =1,inplace = True)

FMSB03_자재마스터.rename(columns={'class_nm_3':'jajae_class_nm_3'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_nm_2':'jajae_class_nm_2'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_nm_1':'jajae_class_nm_1'},inplace = True)

FMSB03_자재마스터.rename(columns={'class_cd_3':'jajae_class_cd_3'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_cd_2':'jajae_class_cd_2'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_cd_1':'jajae_class_cd_1'},inplace = True)


FMSX05_장비마스터.rename(columns={'class_nm_4':'jangbi_class_nm_4'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_3':'jangbi_class_nm_3'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_2':'jangbi_class_nm_2'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_1':'jangbi_class_nm_1'},inplace = True)


FMSX05_장비마스터.rename(columns={'class_cd_4':'jangbi_class_cd_4'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_3':'jangbi_class_cd_3'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_2':'jangbi_class_cd_2'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_1':'jangbi_class_cd_1'},inplace = True)

FMSA10_작업지시_결과.rename(columns={'class_nm_4':'job_class_nm_4'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_3':'job_class_nm_3'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_2':'job_class_nm_2'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_1':'job_class_nm_1'},inplace = True)


FMSA10_작업지시_결과.rename(columns={'class_cd_4':'job_class_cd_4'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_3':'job_class_cd_3'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_2':'job_class_cd_2'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_1':'job_class_cd_1'},inplace = True)

fmsa10 =FMSA10_작업지시_결과[['job_cd','job_nm','job_jisi_dt','job_enddt','job_shm','job_ehm','job_class_nm_1','job_class_nm_2','job_class_nm_3','job_class_nm_4']]
fmsb03 = FMSB03_자재마스터[['mat_cd','mat_nm','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]
fmsx05 = FMSX05_장비마스터[['fac_cd','fac_nm','jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4','fac_instdt']]


FMSB07_자재출고["mat_cd"] = FMSB07_자재출고["mat_cd"].astype("str")
FMSB07_자재출고["mat_cd"] = list(map(lambda x: x[:-3],FMSB07_자재출고["mat_cd"].tolist())) # FMSB03과 inner join하기 위해 mat_cd 형식 일치
fmsb03["mat_cd"] = fmsb03["mat_cd"].astype("str")
jajae = pd.merge(FMSB07_자재출고,fmsb03 ,on = 'mat_cd', how = 'inner')
jajae = jajae[['job_cd','outw_day','mat_cd','outw_qty','mat_nm_x','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]


jangbi = pd.merge(FMSA11_작업장비이력, fmsx05, how='inner', on='fac_cd')
jangbi_broken = pd.merge(fmsa10,jangbi, on='job_cd',how = 'inner')

jangbi_broken.drop(['job_cont','chg_sysdt','chg_emp_no'],axis=1,inplace=True)
jangbi_broken = jangbi_broken.dropna()

jangbi_broken['자재매칭여부'] = jangbi_broken['job_cd']
arr_장비 = list(jangbi_broken['job_cd'])
arr_자재 = list(jajae['job_cd'])
#자재 매칭이 안되는 경우
arr = list(set(arr_장비)- set(arr_자재))

def function(x):
    if x in arr:
        return 0
    else:
         return 1

jangbi_broken['자재매칭여부'] = jangbi_broken['자재매칭여부'].apply(function)
df = pd.merge(jangbi_broken,jajae, on='job_cd',how = 'inner')

jangbi_jajae = df[['job_cd','job_nm','job_jisi_dt','job_enddt','job_shm','job_ehm','job_class_nm_1','job_class_nm_2','job_class_nm_3','job_class_nm_4','fac_cd','fac_nm','fac_instdt','mat_cd','mat_nm_x',
           'jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4','자재매칭여부','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_80773/625519574.py:1: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  FMSA10_작업지시_결과 = pd.read_csv(path+'FMSA10_작업지시_결과_ver2.csv')
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_80773/625519574.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fmsb03["mat_cd"] = fmsb03["mat_cd"].astype("str")


In [12]:
# 기계 = jangbi_jajae[jangbi_jajae['jangbi_class_nm_1']=='기계']
# 통신 = jangbi_jajae[jangbi_jajae['jangbi_class_nm_1']=='통신']
# 전기 = jangbi_jajae[jangbi_jajae['jangbi_class_nm_1']=='전기']
# 방재 = jangbi_jajae[jangbi_jajae['jangbi_class_nm_1']=='방재']
# 승강기 = jangbi_jajae[jangbi_jajae['jangbi_class_nm_1']=='승강기']
#
# 기계_장비 = jangbi_broken[jangbi_broken['jangbi_class_nm_1']=='기계']
# 통신_장비 = jangbi_broken[jangbi_broken['jangbi_class_nm_1']=='통신']
# 전기_장비 = jangbi_broken[jangbi_broken['jangbi_class_nm_1']=='전기']
# 방재_장비 = jangbi_broken[jangbi_broken['jangbi_class_nm_1']=='방재']
# 승강기_장비 = jangbi_broken[jangbi_broken['jangbi_class_nm_1']=='승강기']



In [13]:
jangbi_broken

,job_cd,job_nm,job_jisi_dt,job_enddt,job_shm,job_ehm,job_class_nm_1,job_class_nm_2,job_class_nm_3,job_class_nm_4,...,input_sysdt,input_emp_no,input_scr_nm,fac_nm,jangbi_class_nm_1,jangbi_class_nm_2,jangbi_class_nm_3,jangbi_class_nm_4,fac_instdt,자재매칭여부
0,211001-00047,2동 1구역 430호 남자화장실 비데 전열불량,2021-10-01,2021-10-01,2021-10-01 오후 2:30:00,2021-10-01 오후 2:40:00,2동 1구역,4층,분전반,EPS실,...,2021-10-01 오후 3:24:08,crowonthetree,JOB_A11,LN-4E1(E),전기,전력공급설비,분전반,EPS실,2012-09-01,0
3,211001-00081,1동 1구역 2층 복도내 거실통로유도등 점등불량 교체 작업,2021-10-01,2021-10-01,2021-10-01 오후 1:30:00,2021-10-01 오후 1:45:00,1동 1구역,2층,유도등,거실통로 유도등,...,2021-10-21 오전 10:33:08,skysen83,JOB_A11,1동 거실통로 유도등,방재,피난설비,유도등,거실통로 유도등,2012-05-01,1
4,211001-00070,2-1동 옥상층 66번 CCTV PTZ 동작 불량,2021-10-01,2021-10-01,2021-10-01 오전 9:00:00,2021-10-01 오후 5:31:00,2동 1구역,옥상층,방범시스템,CCTV 카메라,...,2021-10-01 오후 5:39:22,hugh7192,JOB_A11,2동 CCTV #66,통신,방범설비,방범시스템,CCTV 카메라,2019-10-25,1
5,211001-00069,5-3동 1F 34번 얼굴인식단말기 동작 불량,2021-10-01,2021-10-01,2021-10-01 오후 1:10:00,2021-10-01 오후 2:11:00,5동 3구역,1층,출입통제시스템,얼굴인식,...,2021-10-01 오후 5:34:38,sghong923,JOB_A11,5-3동 얼굴인식 #34,통신,방범설비,출입통제시스템,얼굴인식,2019-10-10,0
6,211001-00025,7-1동 6층 남자화장실 대변기 뚜껑 떨어짐,2021-10-01,2021-10-01,2021-10-01 오전 9:20:00,2021-10-01 오전 9:40:00,7-1동,6층,위생설비,대변기,...,2021-10-01 오후 9:33:55,kwak6400,JOB_A11,7-1동-6F 화장실(남)대변기,기계,"위생, 탱크설비",위생설비,대변기,2019-11-27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6825,221011-00020,12동 4-1 남자화장실 소변기 뒤쪽 누수,2022-10-11,2022-10-11,2022-10-11 오전 10:30:00,2022-10-11 오전 11:00:00,12-3동,4층,위생설비,소변기,...,2022-10-11 오후 1:25:55,rhkdtlr761,JOB_A11,소변기 #07,기계,"위생, 탱크설비",위생설비,소변기,2012-03-01,0
6826,221011-00011,13동 5-7 여자화장실 오른쪽 첫번째 대변기 밑 밸브쪽 누수,2022-10-11,2022-10-11,2022-10-11 오전 9:40:00,2022-10-11 오전 10:10:00,13-1동,5층,위생설비,비데,...,2022-10-11 오전 10:38:16,rhkdtlr761,JOB_A11,물내림비데 #30,기계,"위생, 탱크설비",위생설비,비데,2012-03-01,0
6827,221011-00066,13-2동 1층 방재실 R형 수신기 예비전원 불량 교체 (2개),2022-10-11,2022-10-11,2022-10-11 오후 2:30:00,2022-10-11 오후 3:10:00,13-2동,1층,자동화재탐지설비,수신기,...,2022-10-11 오후 4:09:26,kjkim63,JOB_A11,13동 GR형복합식수신기,방재,경보설비,자동화재탐지설비,수신기,2013-11-01,1
6829,221011-00044,16동 1층 동쪽 여자화장실 대변기 막힘,2022-10-11,2022-10-11,2022-10-11 오전 8:32:00,2022-10-11 오전 9:30:00,16동,1층,위생설비,대변기,...,2022-10-11 오후 2:59:13,je0303,JOB_A11,1F여2동쪽 공용대변기(5),기계,"위생, 탱크설비",위생설비,대변기,2014-06-01,0


# 작업 소모 시간

In [14]:
# 오전, 오후 삭제하고 시간에 반영

def time_trans(x):
    if "오후" in x:
        x = datetime.datetime.strptime(x[:10] + x[13:], "%Y-%m-%d %H:%M:%S") + datetime.timedelta(hours=12)
    else:
        x = datetime.datetime.strptime(x[:10] + x[13:], "%Y-%m-%d %H:%M:%S")

    return x

#jangbi_broken = jangbi_broken.drop(jangbi.index[1124]) # job_shm이 Y-M-D만 존재 (H:M:S 없음)
jangbi_broken = jangbi_broken.reset_index().drop("index",axis=1)

jangbi_broken["job_start"] = jangbi_broken["job_shm"].apply(time_trans)
jangbi_broken["job_end"] = jangbi_broken["job_ehm"].apply(time_trans)

jangbi_broken["job_time"] = 0

for time in range(len(jangbi_broken)):
    jangbi_broken.loc[time,"job_time"] = jangbi_broken.loc[time,"job_end"] - jangbi_broken.loc[time,"job_start"]

In [15]:
jangbi_broken

,job_cd,job_nm,job_jisi_dt,job_enddt,job_shm,job_ehm,job_class_nm_1,job_class_nm_2,job_class_nm_3,job_class_nm_4,...,fac_nm,jangbi_class_nm_1,jangbi_class_nm_2,jangbi_class_nm_3,jangbi_class_nm_4,fac_instdt,자재매칭여부,job_start,job_end,job_time
0,211001-00047,2동 1구역 430호 남자화장실 비데 전열불량,2021-10-01,2021-10-01,2021-10-01 오후 2:30:00,2021-10-01 오후 2:40:00,2동 1구역,4층,분전반,EPS실,...,LN-4E1(E),전기,전력공급설비,분전반,EPS실,2012-09-01,0,2021-10-01 14:30:00,2021-10-01 14:40:00,0 days 00:10:00
1,211001-00081,1동 1구역 2층 복도내 거실통로유도등 점등불량 교체 작업,2021-10-01,2021-10-01,2021-10-01 오후 1:30:00,2021-10-01 오후 1:45:00,1동 1구역,2층,유도등,거실통로 유도등,...,1동 거실통로 유도등,방재,피난설비,유도등,거실통로 유도등,2012-05-01,1,2021-10-01 13:30:00,2021-10-01 13:45:00,0 days 00:15:00
2,211001-00070,2-1동 옥상층 66번 CCTV PTZ 동작 불량,2021-10-01,2021-10-01,2021-10-01 오전 9:00:00,2021-10-01 오후 5:31:00,2동 1구역,옥상층,방범시스템,CCTV 카메라,...,2동 CCTV #66,통신,방범설비,방범시스템,CCTV 카메라,2019-10-25,1,2021-10-01 09:00:00,2021-10-01 17:31:00,0 days 08:31:00
3,211001-00069,5-3동 1F 34번 얼굴인식단말기 동작 불량,2021-10-01,2021-10-01,2021-10-01 오후 1:10:00,2021-10-01 오후 2:11:00,5동 3구역,1층,출입통제시스템,얼굴인식,...,5-3동 얼굴인식 #34,통신,방범설비,출입통제시스템,얼굴인식,2019-10-10,0,2021-10-01 13:10:00,2021-10-01 14:11:00,0 days 01:01:00
4,211001-00025,7-1동 6층 남자화장실 대변기 뚜껑 떨어짐,2021-10-01,2021-10-01,2021-10-01 오전 9:20:00,2021-10-01 오전 9:40:00,7-1동,6층,위생설비,대변기,...,7-1동-6F 화장실(남)대변기,기계,"위생, 탱크설비",위생설비,대변기,2019-11-27,0,2021-10-01 09:20:00,2021-10-01 09:40:00,0 days 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6338,221011-00020,12동 4-1 남자화장실 소변기 뒤쪽 누수,2022-10-11,2022-10-11,2022-10-11 오전 10:30:00,2022-10-11 오전 11:00:00,12-3동,4층,위생설비,소변기,...,소변기 #07,기계,"위생, 탱크설비",위생설비,소변기,2012-03-01,0,2022-10-11 10:30:00,2022-10-11 11:00:00,0 days 00:30:00
6339,221011-00011,13동 5-7 여자화장실 오른쪽 첫번째 대변기 밑 밸브쪽 누수,2022-10-11,2022-10-11,2022-10-11 오전 9:40:00,2022-10-11 오전 10:10:00,13-1동,5층,위생설비,비데,...,물내림비데 #30,기계,"위생, 탱크설비",위생설비,비데,2012-03-01,0,2022-10-11 09:40:00,2022-10-11 10:10:00,0 days 00:30:00
6340,221011-00066,13-2동 1층 방재실 R형 수신기 예비전원 불량 교체 (2개),2022-10-11,2022-10-11,2022-10-11 오후 2:30:00,2022-10-11 오후 3:10:00,13-2동,1층,자동화재탐지설비,수신기,...,13동 GR형복합식수신기,방재,경보설비,자동화재탐지설비,수신기,2013-11-01,1,2022-10-11 14:30:00,2022-10-11 15:10:00,0 days 00:40:00
6341,221011-00044,16동 1층 동쪽 여자화장실 대변기 막힘,2022-10-11,2022-10-11,2022-10-11 오전 8:32:00,2022-10-11 오전 9:30:00,16동,1층,위생설비,대변기,...,1F여2동쪽 공용대변기(5),기계,"위생, 탱크설비",위생설비,대변기,2014-06-01,0,2022-10-11 08:32:00,2022-10-11 09:30:00,0 days 00:58:00


In [16]:
jangbi_broken['job_time']

0       0 days 00:10:00
1       0 days 00:15:00
2       0 days 08:31:00
3       0 days 01:01:00
4       0 days 00:20:00
             ...       
6338    0 days 00:30:00
6339    0 days 00:30:00
6340    0 days 00:40:00
6341    0 days 00:58:00
6342    0 days 00:23:00
Name: job_time, Length: 6343, dtype: object

In [17]:
#jangbi_broken[['jangbi_class_nm_4','job_time']].to_csv('장비_작업시간.csv')

In [21]:
time_data = pd.read_csv('장비_작업시간.csv')[['jangbi_class_nm_4','job_time']]
#time_data['job_class_nm_4'].dt.days
# 일
time_data['일'] = time_data['job_time'].str[:-13]
time_data=time_data[~time_data['일'].str.contains('-')]
time_data['시간'] = time_data['job_time'].str[-8:]
time_data['시'] = pd.to_datetime(time_data['시간']).dt.hour
time_data['분'] = pd.to_datetime(time_data['시간']).dt.minute
time_data[['일','시','분']] = time_data[['일','시','분']].astype(int)
time_data['총시간']= time_data['일']*24*60 + time_data['시'] * 60 + time_data['분']


time_final_median = time_data.groupby('jangbi_class_nm_4').median()['총시간']
time_final_median = time_final_median.reset_index()

time_final_mean = time_data.groupby('jangbi_class_nm_4').mean()['총시간']
time_final_mean = time_final_mean.reset_index()

time_final_max = time_data.groupby('jangbi_class_nm_4').max()['총시간']
time_final_max = time_final_max.reset_index()

time_final_min = time_data.groupby('jangbi_class_nm_4').min()['총시간']
time_final_min = time_final_min.reset_index()

In [25]:
pd.concat([time_final_median,time_final_mean,time_final_max,time_final_min],axis =1)

,jangbi_class_nm_4,총시간,jangbi_class_nm_4,총시간,jangbi_class_nm_4,총시간,jangbi_class_nm_4,총시간
0,CCTV 서버,66.0,CCTV 서버,78.820225,CCTV 서버,768,CCTV 서버,20
1,CCTV 저장장치,83.0,CCTV 저장장치,79.823529,CCTV 저장장치,131,CCTV 저장장치,9
2,CCTV 카메라,88.0,CCTV 카메라,144.621145,CCTV 카메라,3605,CCTV 카메라,0
3,DDC 판넬,60.0,DDC 판넬,201.428571,DDC 판넬,390,DDC 판넬,60
4,EPS실,30.5,EPS실,78.000000,EPS실,2130,EPS실,10
...,...,...,...,...,...,...,...,...
146,항온항습기 실외기,150.0,항온항습기 실외기,150.000000,항온항습기 실외기,150,항온항습기 실외기,150
147,항온항습기실,60.0,항온항습기실,190.000000,항온항습기실,450,항온항습기실,60
148,헤드,50.0,헤드,106.500000,헤드,840,헤드,30
149,환기조화기,60.0,환기조화기,96.666667,환기조화기,300,환기조화기,40
